In [585]:
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

In [586]:
## join up all the data so we can play around with visuals
df_2000_1=pd.read_csv('CSV_files/Final_2000_data.csv',index_col=0)
df_2000_2=pd.read_csv('CSV_files/Final_2000_data_sample.csv',index_col=0)

df_2000 = df_2000_1.merge(df_2000_2,how='inner',on='tractid')

# create a column on representing the year of the census data
df_2000['year']='2000-01-01'
df_2000['year']= pd.to_datetime(df_2000['year']) 

df_2010_1 = pd.read_csv('CSV_files/Final_2010_data.csv',index_col=0)
df_2010_2= pd.read_csv('CSV_files/Final_2010_data_sample.csv',index_col=0)

df_2010 = df_2010_1.merge(df_2010_2,how='inner',on='tractid')
df_2010['year']='2010-01-01'
df_2010['year']= pd.to_datetime(df_2010['year']) 

In [587]:
# import changes file too

df_changes_full = pd.read_csv('CSV_files/fullcount_final.csv',index_col=0)
df_changes_sample =  pd.read_csv('CSV_files/samplecount_final.csv',index_col=0)

In [588]:
# before visualizing, remove all the tracts that were removed during EDA process
tracts_to_remove = [25025981201,25025980300,25025981000,25025981202,25025981300,25025981501,25025981502,
                    25025981600,25025981700,25025981800,25025060700,25025980700,25025980101]

In [589]:
df_2010 = df_2010[df_2010['tractid'].isin(tracts_to_remove) == False]
df_2000 = df_2000[df_2000['tractid'].isin(tracts_to_remove) == False]
df_changes_full = df_changes_full[df_changes_full['tractid'].isin(tracts_to_remove) == False]
df_changes_sample = df_changes_sample[df_changes_sample['tractid'].isin(tracts_to_remove) == False]

In [590]:
# also drop 25025990101 bc it didnt exist in 2000, its index 203
df_2010.drop(203,axis=0,inplace=True)

In [591]:
df_plotly_2000= df_plotly_2000[df_plotly_2000['TRTID10'].isin(tracts_to_remove) == False]
df_plotly_2010= df_plotly_2010[df_plotly_2010['tractid'].isin(tracts_to_remove) == False]

In [592]:
df_2010.to_csv('plotly_2010.csv')
df_2000.to_csv('plotly_2000.csv')

In [644]:
cluster_df=pd.read_csv('CSV_files/clusters.csv')

In [684]:
cluster_df.head()

,Unnamed: 0,tractid,cluster,HINC00_PC,MHMVAL00_PC,MRENT00_PC,percent_owneroccupied_00_PC,percent_non-white00_PC,percent_4yrcollege_degree_ormore_PC,cluster_name
0,0,25025000100,1,-5.21,56.78,50.11,-12.52,27.33,-24.91,Becoming more affordable
1,1,25025000201,1,-16.97,100.30,41.78,-6.08,22.63,12.61,Becoming more affordable
2,2,25025000202,1,-24.08,56.41,58.41,-7.61,2.15,11.10,Becoming more affordable
3,3,25025000301,2,25.76,91.09,51.29,4.77,45.58,37.54,Remaining costly
4,4,25025000302,1,-15.31,24.16,15.71,-3.60,32.15,-1.16,Becoming more affordable


In [686]:
cluster_df['HINC00_PC'].loc[25025000100]

KeyError: 25025000100

In [646]:
cluster_and_2000=df_2000.merge(cluster_df,how='inner',on='tractid')

In [710]:
cluster_and_2000.rename(columns={'HINC00_PC':'Percent Change in Median Income','MHMVAL00_PC':'Percent Change in Median Home Value',
                   'MRENT00_PC':'Percent Change in Median Rent','percent_owneroccupied_00_PC':'Percent Change in Owner Occupied Homes',
                    'percent_non-white00_PC':'Percent Change in Non-White Population','percent_4yrcollege_degree_ormore_PC':'Percent Change in Population with College Degree'},inplace=True)

In [712]:
cluster_and_2000.to_csv('CSV_files/clusters_and_2000.csv')

In [711]:
cluster_and_2000.columns[-7:-1]

Index(['Percent Change in Median Income',
       'Percent Change in Median Home Value', 'Percent Change in Median Rent',
       'Percent Change in Owner Occupied Homes',
       'Percent Change in Non-White Population',
       'Percent Change in Population with College Degree'],
      dtype='object')

In [651]:
df1=df_2000.merge(cluster_df,how='inner',on='tractid')

In [667]:
categories=['Median Household Income','Median Home Value','Median Rent', 
         '% Owner Occupied Housing', '% Non-White',
         '% College Degree']
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[55907,32,378607,1117,64,41],
      theta=categories,
      name='County Average',
      marker={
        'color':'orange'
    }
))

fig.add_trace(go.Scatterpolar(
  r=[56486,37,280846,951,49,35],
  theta=categories,
    name='Remaining costly'
))

fig.add_trace(go.Scatterpolar(
      r=[50071,28,289275,976,63,38],
      theta=categories,
      name='Becoming more affordable'
))
fig.add_trace(go.Scatterpolar(
      r=[38815,24,236787,711,69,17],
      theta=categories,
      name='Gentrifying'
))
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
    ),
      
  ),
  showlegend=True
)

fig.show()

In [661]:
df1[['HINC00','percent_owneroccupied_00','MHMVAL00','MRENT00','percent_non-white00','percent_4yrcollege_degree_ormore']].mean()

HINC00                               51003.701258
percent_owneroccupied_00                31.490385
MHMVAL00                            277074.143915
MRENT00                                922.301268
percent_non-white00                     57.858739
percent_4yrcollege_degree_ormore        33.356281
dtype: float64

In [663]:
df2[['hinc12','percent_owneroccupied_10','mhmval12','mrent12','non-white10','pcol12']].mean()

hinc12                       55906.868421
percent_owneroccupied_10        31.991918
mhmval12                    378607.394737
mrent12                       1116.894737
non-white10                     63.681032
pcol12                          41.296526
dtype: float64

In [658]:
df1.groupby('cluster_name')['HINC00','percent_owneroccupied_00','MHMVAL00','MRENT00','percent_non-white00','percent_4yrcollege_degree_ormore'].mean()

,HINC00,percent_owneroccupied_00,MHMVAL00,MRENT00,percent_non-white00,percent_4yrcollege_degree_ormore
cluster_name,,,,,,
Becoming more affordable,50071.892877,28.337898,289275.967114,976.248698,62.758259,38.401296
Gentrifying,38815.015570,24.615605,236787.053914,711.065697,69.162572,16.959610
Remaining costly,56486.609816,37.049649,280846.819569,951.500665,48.984569,34.817631


In [659]:
df2.groupby('cluster_name')['hinc12','percent_owneroccupied_10','mhmval12','mrent12','non-white10','pcol12'].mean()

,hinc12,percent_owneroccupied_10,mhmval12,mrent12,non-white10,pcol12
cluster_name,,,,,,
Becoming more affordable,44872.090909,26.779719,372684.480519,1085.038961,69.633559,39.670130
Gentrifying,50964.000000,28.798752,342251.612903,963.935484,69.784084,36.856129
Remaining costly,68137.439024,38.093474,397913.414634,1204.634146,55.784213,44.502439


In [652]:
df2=df_2010.merge(cluster_df,how='inner',on='tractid')

In [605]:
cluster_and_2000[cluster_and_2000['tractid']==25025020302]

,tractid,state,county,tract,POP00,NHWHT00,NHBLK00,NTV00,ASIAN00,HISP00,HAW00,INDIA00,CHINA00,FILIP00,JAPAN00,KOREA00,VIET00,MEX00,PR00,CUBAN00,HU00,VAC00,OHU00,A18UND00,A60UP00,A75UP00,AGEWHT00,A15WHT00,A60WHT00,AGEBLK00,A15BLK00,A60BLK00,AGEHSP00,A15HSP00,A60HSP00,AGENTV00,A15NTV00,A60NTV00,AGEASN00,A15ASN00,A60ASN00,FAMILY00,FHH00,OWN00,RENT00,GlobD00,GlobG00,percent_white00,percent_black00,percent_asian00,percent_hispanic00,percent_indian00,percent_chinese00,percent_filip00,percent_japan00,percent_korean00,percent_viet00,percent_mex00,percent_pr00,percent_cuban00,percent_vacant_housing00,percent_occupied_housing00,percent_under18_00,percent_60andup_00,percent_75andup_00,percent_owneroccupied_00,percent_renteroccupied_00,percent_non-white00,INCPC00,HINC00,MHMVAL00,MRENT00,percent_foreign_born,percent_naturalized,percent_recent_immigrants(10),percent_other_languages,percent_hs_degree_orless,percent_4yrcollege_degree_ormore,percent_married,percent_unemployed,percent_employed,percent_professional,percent_manufacturing,percent_self_employed,percent_poverty,percent_houses_30yrsago,percent_multiunit_houses,year,Unnamed: 0,level_0,index,cluster,HINC00_PC,MHMVAL00_PC,MRENT00_PC,percent_owneroccupied_00_PC,percent_non-white00_PC,percent_4yrcollege_degree_ormore_PC,cluster_name
35,25025020302,MA,Suffolk County,Census Tract 203.02,1258.999958,904.270691,121.168846,6.850535,142.576767,74.927734,0.0,30.82741,40.032818,8.991327,37.035706,14.557387,4.281585,12.844754,17.126339,1.712634,704.534729,49.666382,654.868347,93.552628,227.780304,88.628799,904.270691,46.241116,205.730148,115.174629,5.56606,8.34909,74.927734,7.278694,4.281585,3.639347,0.214079,0.428158,132.515045,14.343308,6.636456,177.043518,12.416595,198.451447,456.416931,wba,Semi global,71.824521,9.624214,11.324605,5.951369,2.448563,3.179731,0.714164,2.941676,1.156266,0.340078,1.020235,1.360313,0.136031,7.049529,92.950471,7.430709,18.092161,7.039619,28.167731,64.782744,40.741371,58649.948825,66050.205569,354817.26114,1518.293665,19.91158,7.107634,10.491413,23.450703,23.215007,61.234369,41.835061,4.272699,95.727306,76.895306,3.249097,10.731867,11.066072,67.395929,99.48344,2000-01-01,35,35,35,2,34.37,98.61,43.12,9.8,-45.34,41.72,Average


In [606]:
cluster_and_2000[cluster_and_2000['tractid']==25025020303]

,tractid,state,county,tract,POP00,NHWHT00,NHBLK00,NTV00,ASIAN00,HISP00,HAW00,INDIA00,CHINA00,FILIP00,JAPAN00,KOREA00,VIET00,MEX00,PR00,CUBAN00,HU00,VAC00,OHU00,A18UND00,A60UP00,A75UP00,AGEWHT00,A15WHT00,A60WHT00,AGEBLK00,A15BLK00,A60BLK00,AGEHSP00,A15HSP00,A60HSP00,AGENTV00,A15NTV00,A60NTV00,AGEASN00,A15ASN00,A60ASN00,FAMILY00,FHH00,OWN00,RENT00,GlobD00,GlobG00,percent_white00,percent_black00,percent_asian00,percent_hispanic00,percent_indian00,percent_chinese00,percent_filip00,percent_japan00,percent_korean00,percent_viet00,percent_mex00,percent_pr00,percent_cuban00,percent_vacant_housing00,percent_occupied_housing00,percent_under18_00,percent_60andup_00,percent_75andup_00,percent_owneroccupied_00,percent_renteroccupied_00,percent_non-white00,INCPC00,HINC00,MHMVAL00,MRENT00,percent_foreign_born,percent_naturalized,percent_recent_immigrants(10),percent_other_languages,percent_hs_degree_orless,percent_4yrcollege_degree_ormore,percent_married,percent_unemployed,percent_employed,percent_professional,percent_manufacturing,percent_self_employed,percent_poverty,percent_houses_30yrsago,percent_multiunit_houses,year,Unnamed: 0,level_0,index,cluster,HINC00_PC,MHMVAL00_PC,MRENT00_PC,percent_owneroccupied_00_PC,percent_non-white00_PC,percent_4yrcollege_degree_ormore_PC,cluster_name
36,25025020303,MA,Suffolk County,Census Tract 203.03,2806.675935,2015.88147,270.120483,15.27183,317.84494,167.035629,0.0,68.723236,89.244751,20.044277,82.563332,32.452637,9.544893,28.63468,38.179573,3.817957,1570.612183,110.720764,1459.891479,208.555924,507.78833,197.5793,2015.88147,103.084846,458.632141,256.757629,12.408361,18.612543,167.035629,16.226318,9.544893,8.113159,0.477245,0.954489,295.414459,31.975393,14.794585,394.681335,27.680191,442.405823,1017.485657,wba,Semi global,71.824518,9.624213,11.324604,5.951369,2.448563,3.179731,0.714164,2.941677,1.156266,0.340078,1.020235,1.360313,0.136031,7.049529,92.950475,7.430709,18.092161,7.039619,28.167732,64.782743,40.74137,58649.948901,66050.20881,354817.252909,1518.293686,19.91158,7.107635,10.491413,23.450703,23.215004,61.234364,41.835064,4.272698,95.727302,76.895309,3.249097,10.731867,11.066072,67.39593,99.48344,2000-01-01,36,36,36,2,26.72,52.89,66.89,-14.66,39.47,12.99,Average


## EDA

### Racial and ethnic changes

In [211]:
# 3% decline in white population
(df_2010.nhwht10.sum()-df_2000.NHWHT00.sum())/df_2000.NHWHT00.sum()

-0.03519017636997504

In [212]:
# 16% increase in non-white population
((df_2010[['nhblk10','ntv10','asian10','hisp10',
                                                   'haw10','india10','china10','filip10',
                                                   'japan10','korea10','viet10','mex10',
                                                   'pr10','cuban10']].sum().sum())-df_2000[['NHBLK00','NTV00','ASIAN00','HISP00',
                                                   'HAW00','INDIA00','CHINA00','FILIP00',
                                                   'JAPAN00','KOREA00','VIET00','MEX00',
                                                   'PR00','CUBAN00']].sum().sum())/(df_2000[['NHBLK00','NTV00','ASIAN00','HISP00',
                                                   'HAW00','INDIA00','CHINA00','FILIP00',
                                                   'JAPAN00','KOREA00','VIET00','MEX00',
                                                   'PR00','CUBAN00']].sum().sum())

0.1577839994293794

In [ ]:
import plotly.graph_objects as go
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=animals, y=[20, 14, 23]),
    go.Bar(name='LA Zoo', x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [262]:
# bar chart showing breakup of white vs. non -white pop in boston 2000 vs. 2010
import plotly.graph_objects as go



fig = go.Figure(data=[go.Bar(name='White', x=['2000','2010'], y=[df_2000.NHWHT00.sum(),df_2010.nhwht10.sum()]),
                      go.Bar(name='Non-white', x=['2000','2010'], y=[df_2000[['NHBLK00','NTV00','ASIAN00','HISP00',
                                                   'HAW00','INDIA00','CHINA00','FILIP00',
                                                   'JAPAN00','KOREA00','VIET00','MEX00',
                                                   'PR00','CUBAN00']].sum().sum(),
                                          df_2010[['nhblk10','ntv10','asian10','hisp10',
                                                   'haw10','india10','china10','filip10',
                                                   'japan10','korea10','viet10','mex10',
                                                   'pr10','cuban10']].sum().sum()])])
# Change the bar mode
fig.update_layout(title=
                  go.layout.Title(text='Change in Racial Composition of Population 2000-2010',
                                  xref='paper',
                                 x=0),
                  barmode='stack',
                  xaxis=
                  go.layout.XAxis(
                     title=go.layout.xaxis.Title(
                     text='Year'),
                      tickmode='array',
                      tickvals=[2000,2010],
                      ticktext=[2000,2010]),
                  yaxis=go.layout.YAxis(
                      title=go.layout.yaxis.Title(
                      text='Population')
                 ))
fig.show()

In [255]:
df_2000['population_bucket']=df_2000['POP00'].apply(lambda x: 'less than 2500' if x < 2500 else
                                                        ('2500-3500' if x < 3500 else (
                                                        ('3500-4500' if x<4500 else
                                                        '4500-8000'))))

## income

In [337]:
df_changes_sample['HINC00'].describe()

count       191.000000
mean      50846.060078
std       20540.883432
min       15362.751600
25%       37189.964700
50%       47486.250000
75%       63371.983500
max      125404.629855
Name: HINC00, dtype: float64

In [640]:
df_changes_sample['hinc12'].describe()

count       190.000000
mean      55906.868421
std       28220.983846
min       10446.000000
25%       36240.000000
50%       52184.500000
75%       69859.500000
max      176357.000000
Name: hinc12, dtype: float64

In [641]:
(52184-47486)/47486

0.09893442277723961

In [431]:
def func(x):
    if x >= 63371:
        return 'Top 25% (> 63k)'
    elif 37189<x<63371:
        return 'Middle 50% (<63k and >37k)'
    else:
        return 'Bottom 25% (<37k)'

In [432]:
df_changes_sample['income_bracket']=df_changes_sample['HINC00'].apply(func)

In [433]:
# resave to csv
df_changes_sample.to_csv('plotly_changes.csv')

In [365]:
df_changes_sample['income_bracket'].value_counts()

Middle 50% (<63k and >37k)    95
Bottom 25% (<37k)             48
Top 25% (> 63k)               48
Name: income_bracket, dtype: int64

In [373]:
df_changes_sample[(df_changes_sample['income_bracket']=='Top 25% (> 63k)')&(df_changes_sample['HINC00_PC']<0)].describe()

,tractid,INCPC00,HINC00,MHMVAL00,MRENT00,percent_foreign_born,percent_naturalized,percent_recent_immigrants(10),percent_other_languages,percent_hs_degree_orless,percent_4yrcollege_degree_ormore,percent_married,percent_unemployed,percent_employed,percent_professional,percent_manufacturing,percent_self_employed,percent_poverty,percent_houses_30yrsago,percent_multiunit_houses,incpc12,hinc12,mhmval12,mrent12,pfb12,pnat12,p10imm12,polang12,phs12,pcol12,pmar12,punemp12,percent_employed12,pprof12,pmanuf12,psemp12,ppov12,p30old12,pmulti12,INCPC00_PC,HINC00_PC,MHMVAL00_PC,MRENT00_PC,percent_foreign_born_PC,percent_naturalized_PC,percent_recent_immigrants(10)_PC,percent_other_languages_PC,percent_hs_degree_orless_PC,percent_4yrcollege_degree_ormore_PC,percent_married_PC,percent_unemployed_PC,percent_employed_PC,percent_professional_PC,percent_manufacturing_PC,percent_self_employed_PC,percent_poverty_PC,percent_houses_30yrsago_PC,percent_multiunit_houses_PC
count,1.500000e+01,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,2.502507e+10,44047.000084,80727.553867,342493.328787,971.800000,15.433025,7.077633,6.610124,19.263982,26.798447,53.229442,37.889341,3.569311,96.430689,56.306393,5.266610,8.330099,8.125812,92.035402,79.904263,52955.533333,71902.400000,519313.333333,1385.066667,16.867334,7.375333,6.972000,20.982667,23.123333,58.548000,35.433333,5.947333,95.360383,56.460666,4.026000,8.270000,11.53800,93.510667,78.193333,26.051195,-11.238645,57.309406,44.595186,12.360375,19.930136,18.416202,9.175688,-15.965573,14.093173,-6.112294,84.392583,-1.097752,0.997153,-2.787617,0.899336,51.838513,1.678719,-2.230468
std,7.602105e+04,25151.001350,15413.042562,160842.370131,205.447664,8.884702,4.537482,3.846151,9.198539,15.948152,22.930853,9.107074,1.750035,1.750035,14.782504,2.315893,3.818392,4.361086,6.951029,15.827260,26107.487177,15971.393475,197227.837309,215.045001,9.861223,4.698925,4.171521,11.497288,12.913420,21.871951,9.141427,3.505048,3.441256,15.112461,2.177685,4.528196,6.22594,7.505448,16.231482,22.501986,6.306825,24.909989,14.773247,19.863635,64.122978,48.457737,30.847836,28.079011,14.732064,15.066291,109.229514,3.474476,12.752671,82.336794,40.781289,73.389031,5.721542,4.035407
min,2.502500e+10,21440.000000,65414.525400,195000.000000,722.000000,3.108808,0.805987,1.312576,7.719505,3.326457,22.997947,26.566546,0.661455,93.404342,35.350877,0.985915,4.766839,2.881303,72.727273,45.943709,29131.000000,54607.000000,322200.000000,1063.000000,4.090000,2.600000,2.130000,7.650000,0.770000,27.799999,17.360001,0.830000,89.865514,36.349998,1.350000,2.680000,3.55000,67.610001,47.169998,-7.599419,-24.082755,15.867026,15.713135,-20.451865,-52.292631,-38.383205,-21.593853,-81.430952,-1.157077,-36.197701,-53.768867,-7.499858,-18.842003,-79.081279,-53.294700,-34.758252,-7.036249,-9.068823
25%,2.502501e+10,26900.500000,69251.414400,231450.000000,799.000000,7.773926,4.311658,3.174553,13.434899,16.800712,32.605545,31.773408,2.320429,95.314128,43.265269,3.870626,5.728126,5.416763,92.085581,67.140045,34200.000000,59834.500000,366400.000000,1264.000000,9.655000,4.100000,4.185000,13.025000,14.655000,40.485001,29.610000,3.330000,92.976042,44.900000,2.545000,4.885000,6.19000,94.280003,65.199999,6.533803,-15.912885,45.862873,37.301367,0.390132,-9.991052,-12.849214,-16.812895,-25.003771,4.765437,-15.254229,6.546073,-3.505008,-6.024164,-45.572328,-28.820983,5.921186,-0.841181,-3.895194
50%,2.502503e+10,28177.000000,74512.890900,265600.000000,891.000000,13

In [377]:
(71902-80728)/80728

-0.10933009612526014

In [375]:
df_changes_sample[(df_changes_sample['income_bracket']=='Bottom 25% (<37k)')&(df_changes_sample['HINC00_PC']<0)].describe()

,tractid,INCPC00,HINC00,MHMVAL00,MRENT00,percent_foreign_born,percent_naturalized,percent_recent_immigrants(10),percent_other_languages,percent_hs_degree_orless,percent_4yrcollege_degree_ormore,percent_married,percent_unemployed,percent_employed,percent_professional,percent_manufacturing,percent_self_employed,percent_poverty,percent_houses_30yrsago,percent_multiunit_houses,incpc12,hinc12,mhmval12,mrent12,pfb12,pnat12,p10imm12,polang12,phs12,pcol12,pmar12,punemp12,percent_employed12,pprof12,pmanuf12,psemp12,ppov12,p30old12,pmulti12,INCPC00_PC,HINC00_PC,MHMVAL00_PC,MRENT00_PC,percent_foreign_born_PC,percent_naturalized_PC,percent_recent_immigrants(10)_PC,percent_other_languages_PC,percent_hs_degree_orless_PC,percent_4yrcollege_degree_ormore_PC,percent_married_PC,percent_unemployed_PC,percent_employed_PC,percent_professional_PC,percent_manufacturing_PC,percent_self_employed_PC,percent_poverty_PC,percent_houses_30yrsago_PC,percent_multiunit_houses_PC
count,2.600000e+01,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
mean,2.502507e+10,12700.592786,27815.000209,179835.601027,524.010957,26.347626,9.169963,12.940507,41.326548,56.855355,24.923433,26.297089,11.878873,88.121127,29.817837,6.750998,3.991062,35.986618,75.024161,88.981753,16718.769231,21508.307692,324907.769231,704.192308,28.045769,12.390000,13.855769,44.243462,50.082692,28.398846,18.810385,16.559615,84.043271,34.516154,5.180385,3.490385,42.160000,70.418846,88.342692,30.527176,-21.295310,80.078859,33.744746,9.887567,55.020874,26.195640,10.838088,-13.562646,42.226714,-29.034116,79.586474,-4.025871,18.367738,-15.881965,24.653569,19.652163,-1.806526,0.250853
std,3.535341e+04,3787.352493,6398.725980,88127.876284,239.150537,9.737164,5.672491,6.125163,13.650773,22.319269,23.061407,9.011558,6.209828,6.209828,12.229696,3.524164,2.439983,7.841178,16.468160,12.247536,7924.807255,5610.775431,166235.105177,408.679632,9.256793,4.927553,6.877626,12.890602,21.706578,24.375106,9.560462,6.445571,6.400027,15.533556,3.123103,2.601320,9.451024,17.595098,10.030333,36.263889,15.409666,163.150793,36.675453,29.811353,71.958938,76.341155,25.623178,26.354729,74.643637,27.826609,116.134352,11.364454,34.256808,48.160592,144.843726,24.747424,30.735841,11.207126
min,2.502500e+10,7394.136846,15362.751600,306.206098,181.000000,13.836647,3.525836,3.904170,18.918137,16.735046,3.287253,11.951801,1.997474,68.300153,7.235585,1.276360,1.140684,22.867441,33.074487,55.966880,7287.000000,12921.000000,-999.000000,326.000000,8.540000,2.760000,1.130000,22.410000,1.220000,4.460000,4.720000,7.200000,65.922144,4.740000,1.130000,0.000000,28.400000,41.380001,70.599998,-40.516052,-54.613513,-426.250851,-28.247668,-38.279845,-64.711429,-90.797772,-41.852398,-92.792963,-68.782673,-67.135632,-65.520199,-28.645383,-45.238001,-71.905862,-100.000000,-34.936514,-44.392293,-22.661187
25%,2.502505e+10,10510.500000,23641.821000,147074.998725,391.655035,20.432391,5.521078,7.739880,34.069874,44.385273,9.104738,19.524024,8.536854,85.489016,21.532026,4.647674,2.454538,30.902484,73.391561,83.470773,13099.250000,16685.250000,268625.000000,396.000000,21.087501,9.867500,10.342500,32.325001,44.870000,12.965000,11.835000,11.965000,82.101521,25.665000,2.805000,1.702500,36.037501,56.504999,80.225002,12.369705,-32.361217,51.463044,8.621321,-6.046665,17.378120,-23.780539,-3.693682,-24.492176,-8.605353,-55.960564,13.190017,-10.432934,-5.403832,-56.555537,-59.756374,6.560103,-27.291468,-4.460624
50%,2.502508e+10,11447.000000,28938.753900

In [376]:
(21508-27815)/27815

-0.22674815746899155

In [354]:
df_changes_sample['income_bracket'][df_changes_sample['HINC00_PC']<0].value_counts()

Middle 50% (<63k and >37k)    34
Bottom 25% (<37k)             26
Top 25% (> 63k)               15
Name: income_bracket, dtype: int64

In [355]:
len(df_changes_sample[(df_changes_sample['HINC00']>47000) &(df_changes_sample['HINC00_PC']>0)])

65

In [356]:
df_changes_sample[['HINC00','income_bracket']]

,HINC00,income_bracket
0,56514.969000,Middle 50% (<63k and >37k)
1,78896.821500,Top 25% (> 63k)
2,71929.638900,Top 25% (> 63k)
3,71319.282300,Top 25% (> 63k)
4,81992.924658,Top 25% (> 63k)
5,38893.138200,Middle 50% (<63k and >37k)
6,69435.027900,Top 25% (> 63k)
7,49332.516350,Middle 50% (<63k and >37k)
8,50056.838190,Middle 50% (<63k and >37k)
9,50056.839431,Middle 50% (<63k and >37k)


In [358]:
fig=go.Figure()
colorsIdx={'Middle 50% (<63k and >37k)':'red','Top 25% (> 63k)':'blue','Bottom 25% (<37k)':'yellow'}
cols=df_changes_sample.income_bracket.map(colorsIdx)
fig.add_trace(go.Scatter(x=df_changes_sample.HINC00,
                         y=df_changes_sample.HINC00_PC,
                        mode='markers',
                        marker=dict(size=10,color=cols)))
fig.show()

In [285]:
df_changes_sample['HINC00'].describe()

count       191.000000
mean      50846.060078
std       20540.883432
min       15362.751600
25%       37189.964700
50%       47486.250000
75%       63371.983500
max      125404.629855
Name: HINC00, dtype: float64

In [276]:
(52009-47486)/47486

0.09524912605820662

## Housing costs

In [512]:
df_all_changes=df_changes_full.merge(df_changes_sample,how='inner',on='tractid')

In [518]:
df_all_changes.to_csv('plotly_changes_full.csv')

In [637]:
df_2010['mrent12'].describe()

count     190.000000
mean     1116.894737
std       371.192187
min       196.000000
25%       933.250000
50%      1083.500000
75%      1349.500000
max      2001.000000
Name: mrent12, dtype: float64

In [638]:
df_2000['MRENT00'].describe()

count     190.000000
mean      922.301268
std       288.738712
min       229.200300
25%       758.184353
50%       871.566708
75%      1112.444550
max      2004.552900
Name: MRENT00, dtype: float64

In [639]:
(1083.5-871.56)/871.56

0.24317316076919554

In [523]:
len(df_all_changes[(df_all_changes['percent_renteroccupied_00_PC']>0) & (df_all_changes['MHMVAL00_PC'])])

49

In [521]:
103/191 # 53 % of tracts experienced and increase in home value while owner occ went up
# vs. only 49/191 tracts expereicen increase in home while renter occupany went up

0.5392670157068062

In [629]:
df_2010['mhmval12'].describe()

count       190.000000
mean     378607.394737
std      154806.929654
min        -999.000000
25%      299150.000000
50%      337150.000000
75%      425400.000000
max      977500.000000
Name: mhmval12, dtype: float64

In [634]:
df_2000['MHMVAL00'].describe()

count       190.000000
mean     277074.143915
std      117207.467872
min       12661.733700
25%      207419.940000
50%      239077.440000
75%      307584.270000
max      916927.784287
Name: MHMVAL00, dtype: float64

In [636]:
# county's median median home value rose 41%
(337150-239077)/239077

0.41021511897840446

In [481]:
# 95% of tracts increas ein home val
len(df_changes_sample[df_changes_sample['MRENT00_PC']>0])

181

In [380]:
# same with rent val
len(df_changes_sample[df_changes_sample['MHMVAL00_PC']>0])

183

In [482]:
181/190

0.9526315789473684

In [480]:
183/191

0.9581151832460733

## Visualizing the clusters

In [683]:
# Cluster 0 - "gentrified"
import plotly.graph_objects as go

categories=['% Change in Household Income','% Change in Median Home Value','% Change in Median Rent', 
         '% Change in Owner Occupied Housing', '% Change in Non-White Population',
         '% Change in Population with College Degree']
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[8.20,76.14,54.41,6.23,28.04,36.73],
      theta=categories,
      name='Baseline, County Avg. (n=191)',
      marker={
        'color':'orange'
    }
))

fig.add_trace(go.Scatterpolar(
  r=[29.95,87.36,69.79,28.49,11.23,129.64],
  theta=categories,
    name='Cluster 1: Gentrifying (n=31)'
))

fig.add_trace(go.Scatterpolar(
      r=[-12.11,72.58,40.70,-7.45,22.16,2.05],
      theta=categories,
      name='Cluster 2: Becoming more affordable (n=77)'
))
fig.add_trace(go.Scatterpolar(
      r=[19.24,81.35,61.29,7.34,40.02,34.12],
      theta=categories,
      name='Cluster 3: Remaining costly (n=82)'
))
fig.update_layout(
    legend_orientation="h",
    legend={'x':-.1,'y':1.2
        
    },
  polar=dict(
    radialaxis=dict(
      visible=True,
    ),

  ),
  showlegend=True,
   
)

fig.show()
legend=dict(x=0, y=1.2)

In [617]:
# Cluster 1-"affordable/low income"
categories=['% Change in Household Income','% Change in Median Home Value','% Change in Median Rent', 
         '% Change in Owner Occupied Housing', '% Change in Non-White Population',
         '% Change in Population with College Degree']
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
  r=[-12.11,72.58,40.70,-7.45,22.16,2.05],
  theta=categories,
    name='Cluster 2: Affordable (n=77)'
))

fig.add_trace(go.Scatterpolar(
      r=[8.20,76.14,54.41,6.23,28.04,36.73],
      theta=categories,
      name='Cluster 2: Affordable (n=77)'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
        range=[-10,120]
    ),
  ),
  showlegend=True
)

fig.show()


In [618]:
# Cluster 2 - "stable, in line with average"
categories=['% Change in Household Income','% Change in Median Home Value','% Change in Median Rent', 
         '% Change in Owner Occupied Housing', '% Change in Non-White Population',
         '% Change in Population with College Degree']
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
  r=[19.24,81.35,61.29,7.34,40.02,34.12],
  theta=categories,
    name='Cluster 3: Gentrified (n=31)'
))

fig.add_trace(go.Scatterpolar(
      r=[8.20,76.14,54.41,6.23,28.04,36.73],
      theta=categories,
      name='Baseline'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
    ),
  ),
  showlegend=False
)

fig.show()

In [173]:
len(df_2000[df_2000['percent_white00']>75])/len(df_2000)

0.3054187192118227

In [50]:
import plotly.io as pio

population = df_joined.POP00
percent_white = df_joined.percent_white00

data = [dict(
  type = 'scatter',
  x = population,
  y = percent_white,
  mode = 'markers',
  transforms = [dict(
    type = 'filter',
    target = 'y',
    operation = '>',
    value = 4
  )]
)]

layout = dict(
    title = 'Scores > 4'
)

fig_dict = dict(data=data, layout=layout)

pio.show(fig_dict, validate=False)

In [51]:
import plotly.graph_objects as go

# Generate dataset
import numpy as np
np.random.seed(1)

x0 = np.random.normal(2, 0.4, 400)
y0 = np.random.normal(2, 0.4, 400)
x1 = np.random.normal(3, 0.6, 600)
y1 = np.random.normal(6, 0.4, 400)
x2 = np.random.normal(4, 0.2, 200)
y2 = np.random.normal(4, 0.4, 200)

# Create figure
fig = go.Figure()

# Add traces
fig.add_trace(
    go.Scatter(
        x=x0,
        y=y0,
        mode="markers",
        marker=dict(color="DarkOrange")
    )
)

fig.add_trace(
    go.Scatter(
        x=x1,
        y=y1,
        mode="markers",
        marker=dict(color="Crimson")
    )
)

fig.add_trace(
    go.Scatter(
        x=x2,
        y=y2,
        mode="markers",
        marker=dict(color="RebeccaPurple")
    )
)

# Add buttons that add shapes
cluster0 = [go.layout.Shape(type="circle",
                            xref="x", yref="y",
                            x0=min(x0), y0=min(y0),
                            x1=max(x0), y1=max(y0),
                            line=dict(color="DarkOrange"))]
cluster1 = [go.layout.Shape(type="circle",
                            xref="x", yref="y",
                            x0=min(x1), y0=min(y1),
                            x1=max(x1), y1=max(y1),
                            line=dict(color="Crimson"))]
cluster2 = [go.layout.Shape(type="circle",
                            xref="x", yref="y",
                            x0=min(x2), y0=min(y2),
                            x1=max(x2), y1=max(y2),
                            line=dict(color="RebeccaPurple"))]

fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(buttons=list([
            dict(label="None",
                 method="relayout",
                 args=["shapes", []]),
            dict(label="Cluster 0",
                 method="relayout",
                 args=["shapes", cluster0]),
            dict(label="Cluster 1",
                 method="relayout",
                 args=["shapes", cluster1]),
            dict(label="Cluster 2",
                 method="relayout",
                 args=["shapes", cluster2]),
            dict(label="All",
                 method="relayout",
                 args=["shapes", cluster0 + cluster1 + cluster2])
        ]),
        )
    ]
)

# Update remaining layout properties
fig.update_layout(
    title_text="Highlight Clusters",
    showlegend=False,
)

fig.show()